In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import launch_exp
from copy import deepcopy
from collections import defaultdict

import yaml

try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

In [ ]:
for expname in ["glob_synth", "glob_synth_rmv_data", "glob_synth_rmv_robdd"]:
    filename = expname + ".yaml"
    with open(filename) as f:
        base_exp = yaml.load(f, Loader)
    for datasetname, remove_col in [("Titanic", "Sex")]: #("Synthetic", "1"), ("Compas", "race_Caucasian"), 
        current_exp = deepcopy(base_exp)
        current_exp["dataset"]["name"] = datasetname

        if datasetname != "Synthetic":
             current_exp["dataset"]["args"] = {"discretizing": False, "hot_encoding": False}

        if expname.endswith("data"):
            current_exp["dataset"]["args"]["remove"] = remove_col
        elif expname.endswith("robdd"):
            current_exp["robdd"]["remove"] = remove_col

        print(expname, datasetname)
        results = []
        for _ in range(10):
            with open("tmp_exp.yaml", "w") as f:
                yaml.dump(current_exp, f, Dumper)
            results.append(launch_exp.main("tmp_exp.yaml"))
            current_exp["seed"] += 1 # can be changed
        
        results_lists = defaultdict(list)
        for result in results:
            for key, value in result.items():
                results_lists[key].append(value)
        results_avg = {key: sum(values) / len(values) for key, values in results_lists.items()}
        print(results_avg)


__Synthetic__ (removed feature : '1')
| remove feature | precision | up coverage | down coverage |
|----------------|:---------:|:-----------:|:-------------:|
| none           |    0.72   |     0.69    |      0.81     |
| dataset        |    0.65   |     0.65    |      0.72     |
| robdd          |    0.68   |     0.55    |      0.72     |

__Compas__ (removed feature : 'race_Caucasian')
| remove feature | precision | up coverage | down coverage |
|----------------|:---------:|:-----------:|:-------------:|
| none           |    0.66   |     0.66    |      0.67     |
| dataset        |    0.65   |     0.65    |      0.68     |
| robdd          |    0.66   |     0.66    |      0.67     |

__Titanic__ (removed feature : 'Sex')
| remove feature | precision | up coverage | down coverage |
|----------------|:---------:|:-----------:|:-------------:|
| none           |    0.72   |     0.67    |      0.68     |
| dataset        |    0.65   |     0.65    |      0.67     |
| robdd          |    0.69   |     0.64    |      0.30     |